Projet kaggle : Tweet Sentiment Extraction

Emma BEUZE, Nacim BOUIA, Antoine FRANCESCHI, Ayoub RHIM, Guillaume RUSSIAS

In [ ]:
#???
import jax
import jax.numpy as jnp
import gzip
import pickle

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
#besoin d'une version > à 1.2.0
pd.__version__

Il y a 4 types de colonnes :


*   textID - unique ID for each piece of text 
*   text - the text of the tweet 
*   sentiment - the general sentiment of the tweet        => il y a 3 sentiments : positive, neutral, negative
*   selected_text - [train only] the text that supports the tweet's sentiment 


**OBJECTIF :  examiner le sentiment donné pour un tweet donné et déterminer le mot ou la phrase qui le soutient le mieux.**






In [ ]:
os.chdir( os.getcwd() + "/donnees" )

Page qui résume comment utiliser les pandas : https://ledatascientist.com/manipulez-vos-donnees-avec-pandas/

In [ ]:
# Load the dataset
train_set = pd.read_csv(r"train.csv")
test_set = pd.read_csv(r"test.csv")

#print(train_set.shape)    => (27481, 4)  (textID, text, selected_text, sentiment )
#print(test_set.shape)     => (3534, 3)   (textID, text, sentiment)

train_data_text = train_set["text"]
train_data_sentiment = train_set["sentiment"]

N_train = train_data_text.shape[0]

train_data = [(train_data_text[k],train_data_sentiment[k]) for k in range(N_train)]
train_labels = train_set["selected_text"]


# check data makes sense

print("Exemple 1 :",train_data[1], "extrait choisi :", train_labels[1])
print("Exemple 2 :",train_data[59], "extrait choisi :", train_labels[59])
print("Exemple 3 :",train_data[750], "extrait choisi :", train_labels[750])

Dans le concours, la métrique à utiliser est le score de Jaccard : </br>
https://en.wikipedia.org/wiki/Jaccard_index </br>
https://towardsdatascience.com/overview-of-text-similarity-metrics-3397c4601f50.


In [ ]:
class RetardedClassifier():
    def __init__(self, n = 2000):
        self._train_frame = None
        self._train_extracts = None
        self._depth = n
    
    def fit(self, train_set):
        self._train_frame = train_set.loc[1:self._depth,("textID","text","sentiment")]
        self._train_extracts = train_set.loc[1:self._depth,("textID","selected_text")]
    
    def __call__(self, new_frame):
        #TODO :)
        return

In [ ]:
R = RetardedClassifier()
R.fit(train_set)
R._train_frame
test_set.head()


In [ ]:
aux = R._train_frame.merge(test_set, how='cross')
aux.head()

In [ ]:
def jaccard(str1, str2): 
    try:
        a = set(str1.lower().split()) 
        b = set(str2.lower().split())
        c = a.intersection(b)
        return float(len(c)) / (len(a) + len(b) - len(c))
    except:
        print(str1, str2)

In [ ]:
str1 = "je m'appelle Emma blablabla"
str2 = "je m'appelle Emma blablabla"
jaccard(str1,str2)

In [ ]:
def row_jac(row):
    return jaccard(row.text_x, row.text_y)

aux2 = aux.loc[:,["text_x","text_y"]]
aux2["score"] = aux.apply(row_jac, axis = 1) 

In [ ]:
aux2

In [ ]:
aux3 = aux2.sort_values('score', ascending=False).drop_duplicates(['text_x'])

In [ ]:
aux3[aux3.score == 0]